XGBoost
===

In [1]:
import xgboost as xgb
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
import scipy as sc
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV

Algorithms
---

In [11]:
# Parsing
#train_set = pd.read_csv('https://media.githubusercontent.com/media/riccamastellone/BIP/master/dataset/output/dataset_polimi_extended.csv?token=AJr57459zM29HNDCIgJxY0xuGQucxIHJks5XaBFfwA%3D%3D',delimiter=',')
#train_set = pd.read_csv('https://media.githubusercontent.com/media/riccamastellone/BIP/master/dataset/output/dataset_polimi_extended.csv?token=AI7lzqjq-gI8IohNvPk8aIYYO_gE6e2gks5XbAPmwA%3D%3D',delimiter=',')
train_set = pd.read_csv('https://raw.githubusercontent.com/riccamastellone/BIP/master/dataset/output/dataset_polimi_extended_with_province_no2014.csv?token=AI7lzlmydW5L8qrMHYvMDbIktyyBkve_ks5Xd4TcwA%3D%3D',delimiter=',')
#test_set = pd.read_csv('https://raw.githubusercontent.com/riccamastellone/BIP/master/dataset/output/test_with_features.csv?token=AJr572V5PSQQk_et0pEphHlyIPeQGxChks5XcT0gwA%3D%3D',delimiter=',')
#test_set = pd.read_csv('https://raw.githubusercontent.com/riccamastellone/BIP/master/dataset/output/test_with_features.csv?token=AI7lzlG4mUzm9mH8ZJUCXSvymRM5i0kdks5XdTB1wA%3D%3D',delimiter=',')
test_set = pd.read_csv("https://raw.githubusercontent.com/riccamastellone/BIP/master/dataset/output/test_with_features_with_provincie.csv?token=AI7lzuSM4N6yF0InDcQgdbssWOQPAPTQks5Xd4UewA%3D%3D",delimiter=',')
pd.options.mode.chained_assignment = None
#Id_ts = test_['ID']
#test_set.drop('ID',axis=1,inplace=True)

In [12]:
train_set.head()

,zona,area,sottoarea,categoria,timestamp,giorno_della_settimana,giorno_del_mese,giorno_dell_anno,mese,anno,...,provincia_TR,provincia_TV,provincia_UD,provincia_VA,provincia_VC,provincia_VE,provincia_VI,provincia_VR,provincia_VT,provincia_VV
0,1,1,5,1,1420070400,3,3,1,1,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,5,1,1420156800,4,4,2,1,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,1,5,1,1420243200,5,5,3,1,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,5,1,1420329600,6,6,4,1,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,5,1,1420416000,0,0,5,1,2015,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
test_set.head()

,zona,area,sottoarea,categoria,timestamp,giorno_della_settimana,giorno_del_mese,giorno_dell_anno,mese,anno,...,provincia_TR,provincia_TV,provincia_UD,provincia_VA,provincia_VC,provincia_VE,provincia_VI,provincia_VR,provincia_VT,provincia_VV
0,10,26,67,1,1463702400,4,4,141,5,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10,26,67,2,1463702400,4,4,141,5,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,26,67,1,1463788800,5,5,142,5,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10,26,67,2,1463788800,5,5,142,5,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,26,67,1,1463875200,6,6,143,5,2016,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145440 entries, 0 to 145439
Columns: 130 entries, zona to provincia_VV
dtypes: float64(113), int64(17)
memory usage: 144.3 MB


In [6]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2880 entries, 0 to 2879
Columns: 129 entries, zona to provincia_VV
dtypes: float64(113), int64(16)
memory usage: 2.8 MB


In [14]:
y_train = train_set.pop('vendite')

In [15]:
cv_params = {'subsample': [0.7,0.9], 'learning_rate':[0.1,0.01]}
ind_params = {'max_depth':40,'min_child_weight': 36,'n_estimators': 500, 'seed':0, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', "nthread":8}

optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params),
                            cv_params,
                            scoring = 'accuracy',
                            cv = 3, 
                            n_jobs = -1)

In [ ]:
optimized_GBM.fit(train_set,y_train)

In [17]:
optimized_GBM.grid_scores_

[mean: 0.24200, std: 0.01815, params: {'subsample': 0.7, 'learning_rate': 0.1},
 mean: 0.24016, std: 0.01589, params: {'subsample': 0.9, 'learning_rate': 0.1},
 mean: 0.24660, std: 0.01611, params: {'subsample': 0.7, 'learning_rate': 0.01},
 mean: 0.24426, std: 0.01585, params: {'subsample': 0.9, 'learning_rate': 0.01}]

In [27]:
optimized_GBM.grid_scores_

[mean: 0.23804, std: 0.01847, params: {'max_depth': 20, 'min_child_weight': 15},
 mean: 0.23888, std: 0.01809, params: {'max_depth': 20, 'min_child_weight': 23},
 mean: 0.23982, std: 0.01825, params: {'max_depth': 20, 'min_child_weight': 30},
 mean: 0.24055, std: 0.01800, params: {'max_depth': 20, 'min_child_weight': 36},
 mean: 0.23947, std: 0.01852, params: {'max_depth': 27, 'min_child_weight': 15},
 mean: 0.23969, std: 0.01951, params: {'max_depth': 27, 'min_child_weight': 23},
 mean: 0.23954, std: 0.01913, params: {'max_depth': 27, 'min_child_weight': 30},
 mean: 0.24077, std: 0.01718, params: {'max_depth': 27, 'min_child_weight': 36},
 mean: 0.23880, std: 0.01710, params: {'max_depth': 35, 'min_child_weight': 15},
 mean: 0.23902, std: 0.01720, params: {'max_depth': 35, 'min_child_weight': 23},
 mean: 0.24024, std: 0.01795, params: {'max_depth': 35, 'min_child_weight': 30},
 mean: 0.24097, std: 0.01906, params: {'max_depth': 35, 'min_child_weight': 36},
 mean: 0.23973, std: 0.01706

In [132]:
optimized_GBM.grid_scores_

[mean: 0.23313, std: 0.03200, params: {'max_depth': 7, 'min_child_weight': 5},
 mean: 0.23312, std: 0.03298, params: {'max_depth': 7, 'min_child_weight': 7},
 mean: 0.23236, std: 0.03461, params: {'max_depth': 7, 'min_child_weight': 9},
 mean: 0.23517, std: 0.03218, params: {'max_depth': 9, 'min_child_weight': 5},
 mean: 0.23539, std: 0.03347, params: {'max_depth': 9, 'min_child_weight': 7},
 mean: 0.23425, std: 0.03682, params: {'max_depth': 9, 'min_child_weight': 9},
 mean: 0.23394, std: 0.03378, params: {'max_depth': 11, 'min_child_weight': 5},
 mean: 0.23414, std: 0.03398, params: {'max_depth': 11, 'min_child_weight': 7},
 mean: 0.23394, std: 0.03382, params: {'max_depth': 11, 'min_child_weight': 9}]

In [ ]:
def custom_evaluation(preds, dtrain):
    labels = dtrain.get_label()
    # return a pair metric_name, result
    # since preds are margin(before logistic transformation, cutoff at 0)
    return 'error', float(sum(labels != (preds > 0.0))) / len(labels)

In [ ]:
cv_params = {'max_depth': [7,9,11], 'min_child_weight': [5,7,9]}
ind_params = {'max_depth': [7,9,11], 'min_child_weight': [5,7,9],'learning_rate': 0.1, 'n_estimators': 500, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', "nthread":8}

optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params),
                            cv_params,
                            scoring = 'accuracy',
                            cv = 2, 
                            n_jobs = -1)

In [19]:
xgdmat = xgb.DMatrix(train_set, y_train, missing=np.nan)

In [11]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':7, 'min_child_weight':5, "nthread":8} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'], 
                early_stopping_rounds = 100)

Will train until cv error hasn't decreased in 100 rounds.


KeyboardInterrupt: 

In [20]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 100)

In [21]:
testdmat = xgb.DMatrix(test_set, missing=np.nan)

In [22]:
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred

array([ 6.25132895,  3.41447377,  1.84642565, ...,  1.84321249,
        0.92337108,  1.14182711], dtype=float32)

In [23]:
y_pred[:10]

array([ 6.25132895,  3.41447377,  1.84642565,  1.95581186,  0.98951685,
        1.25442624,  6.25132895,  2.83298182,  6.25132895,  3.3680203 ], dtype=float32)

In [14]:
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred

array([ 6.83515167,  3.75366306,  2.23812723, ...,  2.07126808,
        1.40312445,  1.3314755 ], dtype=float32)